
**Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.**

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

**PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:**

* URL: https://sql.springboard.com/
* Username: student
* Password: learn_sql@springboard

***

**The data you need is in the "country_club" database. This database
contains 3 tables:**
    * i) the "Bookings" table,
    * ii) the "Facilities" table, and
    * iii) the "Members" table.

***

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables.



### Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

In [ ]:
SELECT *
FROM `Facilities`
WHERE membercost !=0
LIMIT 0 , 30

### Q2: How many facilities do not charge a fee to members?

In [ ]:
SELECT COUNT( name )
FROM `Facilities`
WHERE membercost =0

### Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [ ]:
SELECT facid, name, membercost, monthlymaintenance
FROM `Facilities`
WHERE membercost !=0
AND membercost < 0.2 * monthlymaintenance
LIMIT 0 , 30

### Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

In [ ]:
SELECT *
FROM Facilities
WHERE facid
IN ( 1, 5 )
LIMIT 0 , 30

### Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than 100. Return the name and monthly maintenance of the facilities in question.

In [ ]:
SELECT name, (

    CASE WHEN monthlymaintenance >100
    THEN 'expensive'
    ELSE 'cheap'
    END
) AS Cheap_Or_Expensive
FROM Facilities
LIMIT 0 , 30

### Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

In [ ]:
SELECT firstname, surname
FROM `Members`
WHERE joindate = (
SELECT MAX( joindate )
FROM `Members` )

### Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [ ]:
SELECT DISTINCT (
    CONCAT( m.firstname, ' ', m.surname )
) AS member_name
FROM `Bookings` AS b
LEFT JOIN `Facilities` AS f ON b.facid = f.facid
LEFT JOIN `Members` AS m ON b.memid = m.memid
WHERE f.name
IN (
'Tennis Court 1', 'Tennis Court 2'
)
ORDER BY member_name
LIMIT 0 , 30

### Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than 30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [ ]:
SELECT f.name AS facility_name, CONCAT( m.firstname, ' ', m.surname ) AS member_name,
CASE WHEN b.memid =0
THEN b.slots * f.guestcost
ELSE b.slots * f.membercost
END AS cost
FROM `Bookings` AS b
INNER JOIN `Facilities` AS f ON b.facid = f.facid
INNER JOIN `Members` AS m ON b.memid = m.memid
WHERE LEFT( b.starttime, 10 ) = '2012-09-14'
AND (
(
b.memid =0
AND b.slots * f.guestcost >30
)
OR (
b.memid !=0
AND b.slots * membercost >30
)
)
ORDER BY cost DESC
LIMIT 0 , 30

### Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
SELECT j. *
FROM (

SELECT f.name AS facility_name, (
m.firstname || ' ' || m.surname
) AS member_name,
CASE WHEN b.memid =0
THEN b.slots * f.guestcost
ELSE b.slots * f.membercost
END AS cost
FROM `Bookings` AS b
INNER JOIN `Facilities` AS f ON b.facid = f.facid
INNER JOIN `Members` AS m ON b.memid = m.memid
WHERE LEFT( b.starttime, 10 ) = '2012-09-14'
AND (
   (
        b.memid =0
        AND b.slots * f.guestcost >30
    )
    OR (
        b.memid !=0
        AND b.slots * membercost >30
        )
    )
)j
ORDER BY cost DESC

## Part 2 (Q10, 11, 12, 13)

In [17]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn


'''
Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

'''

def question_10(conn):

    cur = conn.cursor()
    
    query1 = """ 
    
    SELECT name,revenue
    FROM (SELECT name,
    SUM(CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots END) AS revenue
    FROM Bookings B
    INNER JOIN Facilities F
    ON B.facid = F.facid
    GROUP BY name) 
    WHERE revenue < 1000
    ORDER BY revenue
        
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

        
'''
Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

'''      

def question_11(conn):

    cur = conn.cursor()
    
    query2 = """
        
    SELECT M.surname ||','|| M.firstname AS member_name, R.surname ||','|| R.firstname AS recommended_by
    FROM Members M
    INNER JOIN Members R
    ON M.recommendedby = R.memid
    ORDER BY M.surname ||','|| M.firstname
        
        """
    cur.execute(query2)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

        
'''        
Q12: Find the facilities with their usage by member, but not guests        

'''

def question_12(conn):

    cur = conn.cursor()
    
    query3 = """
        
    SELECT name,usage_count
    FROM (SELECT name,
    COUNT(CASE WHEN memid > 0 THEN 1 * slots END) AS usage_count
    FROM Bookings B
    INNER JOIN Facilities F
    ON B.facid = F.facid
    GROUP BY name) 
    ORDER BY usage_count
       
       """
    cur.execute(query3)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

        
'''        
Q13: Find the facilities usage by month, but not guests

'''

def question_13(conn):

    cur = conn.cursor()
    
    query4 = """
       
    SELECT name, usage_month, count
    FROM (SELECT name,
    strftime('%m',starttime) AS usage_month,
    COUNT(slots) AS count
    FROM Bookings B
    INNER JOIN Facilities F
    ON B.facid = F.facid
    GROUP BY name, strftime('%m',starttime)) 
    ORDER BY usage_month
        
        """
    cur.execute(query4)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("#10:")
        question_10(conn)
        
        print("#11:")
        question_11(conn)
        
        print('#12:')
        question_12(conn)
 
        print('#13:')
        question_13(conn)
 
if __name__ == '__main__':
    main()

2.6.0
#10:
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)
#11:
('Bader,Florence', 'Stibbons,Ponder')
('Baker,Anne', 'Stibbons,Ponder')
('Baker,Timothy', 'Farrell,Jemima')
('Boothe,Tim', 'Rownam,Tim')
('Butters,Gerald', 'Smith,Darren')
('Coplin,Joan', 'Baker,Timothy')
('Crumpet,Erica', 'Smith,Tracy')
('Dare,Nancy', 'Joplette,Janice')
('Genting,Matthew', 'Butters,Gerald')
('Hunt,John', 'Purview,Millicent')
('Jones,David', 'Joplette,Janice')
('Jones,Douglas', 'Jones,David')
('Joplette,Janice', 'Smith,Darren')
('Mackenzie,Anna', 'Smith,Darren')
('Owen,Charles', 'Smith,Darren')
('Pinker,David', 'Farrell,Jemima')
('Purview,Millicent', 'Smith,Tracy')
('Rumney,Henrietta', 'Genting,Matthew')
('Sarwin,Ramnaresh', 'Bader,Florence')
('Smith,Jack', 'Smith,Darren')
('Stibbons,Ponder', 'Tracy,Burton')
('Worthington-Smyth,Henry', 'Smith,Tracy')
#12:
('Massage Room 2', 27)
('Squash Court', 195)
('Tennis Court 2', 276)
('Tennis Court 1', 308)
('Badminton Court', 344)
('Table Tennis', 3